## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
# 設定
input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model_xception = keras.applications.xception.Xception(include_top=False,
                                            weights='imagenet',
                                            input_tensor=input_tensor,
                                            pooling=None,
                                            )

'''Resnet 50 架構'''
model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=10)
model_xception.summary()
model.summary()


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

## 添加層數

In [3]:
x = model.output

'''可以參考Cifar10實作章節'''
# x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
# x = Dense(units=128, activation='relu')(x)
# x = Dropout(rate=0.25)(x)
# 要記得展平
x = Flatten()(x)

# 輸出層要自己改
predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 177


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  # This is added back by InteractiveShellApp.init_path()


In [4]:
x_xception = model_xception.output

# x_xception = MaxPooling2D(pool_size=(2, 2), padding='same')(x_xception)
# x_xception = Dense(units=128, activation='relu')(x_xception)
# x_xception = Dropout(rate=0.25)(x_xception)
# 要記得展平
x_xception = Flatten()(x_xception)

# 輸出層改成class=10
predictions_xception = Dense(output_dim=10, activation='softmax')(x_xception)
model_xception = Model(inputs=model_xception.input, outputs=predictions_xception)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  # Remove the CWD from sys.path while we load stuff.


In [5]:
model_xception.summary()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

## 鎖定特定幾層不要更新權重

In [0]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

for layer in model_xception.layers[:100]:
    layer.trainable = False
for layer in model_xception.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [7]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
    mean = np.mean(X_train,axis=(0,1,2,3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train-mean)/(std+1e-7)
    X_test = (X_test-mean)/(std+1e-7)
    return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


In [8]:
print(y_train.shape)
y_train[0]

(50000, 10)


array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

## Training

In [9]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, batch_size=32,epochs=20, validation_data=(x_test, y_test))  # 原本設定是100，但這樣跑太久了



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 98s 2ms/step - loss: 1.6869 - acc: 0.5613 - val_loss: 9.9589 - val_acc: 0.1062
Epoch 2/20
50000/50000 [==============================] - 87s 2ms/step - loss: 1.2427 - acc: 0.6188 - val_loss: 6.4328 - val_acc: 0.1001
Epoch 3/20
50000/50000 [==============================] - 88s 2ms/step - loss: 0.8798 - acc: 0.6986 - val_loss: 4.7240 - val_acc: 0.1350
Epoch 4/20
50000/50000 [==============================] - 88s 2ms/step - loss: 0.7872 - acc: 0.7349 - val_loss: 4.9347 - val_acc: 0.1018
Epoch 5/20
50000/50000 [==============================] - 88s 2ms/step - loss: 0.6677 - acc: 0.7736 - val_loss: 5.3880 - val_acc: 0.1293
Epoch 6/20
50000/50000 [==============================] - 88s 2ms/step - loss: 0.5665 - acc: 0.8087 - val_loss: 5.8528 - val_acc: 0.1123
Epoch 7/20
50000/50000 [======

In [10]:
# compile the model (should be done *after* setting layers to non-trainable)
model_xception.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_xception.fit(x_train, y_train, batch_size=32,epochs=20, validation_data=(x_test, y_test)) # 原本設定是100，但這樣跑太久了

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 54s 1ms/step - loss: 1.5661 - acc: 0.4330 - val_loss: 2.7567 - val_acc: 0.3073
Epoch 2/20
50000/50000 [==============================] - 48s 953us/step - loss: 1.0122 - acc: 0.6512 - val_loss: 1.9194 - val_acc: 0.4647
Epoch 3/20
50000/50000 [==============================] - 48s 959us/step - loss: 0.8790 - acc: 0.6962 - val_loss: 2.1752 - val_acc: 0.4404
Epoch 4/20
50000/50000 [==============================] - 48s 950us/step - loss: 0.7921 - acc: 0.7248 - val_loss: 2.4524 - val_acc: 0.4412
Epoch 5/20
50000/50000 [==============================] - 47s 949us/step - loss: 0.7042 - acc: 0.7531 - val_loss: 2.3366 - val_acc: 0.4760
Epoch 6/20
50000/50000 [==============================] - 47s 950us/step - loss: 0.6228 - acc: 0.7831 - val_loss: 2.4310 - val_acc: 0.4535
Epoch 7/20
50000/50000 [==============================] - 48s 951us/step - loss: 0.5527 - acc: 0.8085 - val_loss: 2.28

In [0]:
# 可以看出在epoch只有20的情況下，xception表現得較好
# 但其實還是要看真實test資料。
# 加入validate_data來進行真正的test
# 在我自己加入幾層的情況下, validate acc遲遲沒有上升，可能是加入的並不好??
  # 兩個模型比起來xception validate acc高出許多, 且有慢慢成長，可能是比較適合的model
  # 是不是